In [63]:
# This code is adapted from [How I Created an AI Version of Myself] by Keith McNulty
# Original repository: https://github.com/keithmcnulty/rag_gemma_regression_book/blob/main/rag_gemma_experiment_open.ipynb
# Modifications by Aleksand Botvin
# I have made the following modifications:
#  - Read PDF files
#  - Using serverless LLM
#  - Creating a Telegram bot as an interface

In [61]:
# Устанавливаем необходимые библиотеки (если они ещё не установлены):
# pip install pymupdf
# pip install langchain-community
# pip install chromadb
# pip install sentence-transformers
# pip install --upgrade torch torchvision torchaudio

import fitz               # PyMuPDF для работы с PDF
import pandas as pd       # Для работы с датафреймами
import re                 # Для работы с регулярными выражениями

from langchain_community.document_loaders import DataFrameLoader  # Загрузчик для работы с DataFrame
from langchain.text_splitter import RecursiveCharacterTextSplitter   # Разбиватель текста на части

import chromadb            # Для работы с базой данных векторных эмбеддингов
from chromadb.utils import embedding_functions
from chromadb.utils.batch_utils import create_batches

import uuid                # Для генерации уникальных идентификаторов

In [39]:
rethinking_directory = "McElreath R. Statistical rethinking a bayesian course second edition.pdf" 
doingbayesianda_directory = "Krushcke J.K. DoingBayesian DataAnalysis.pdf" 
doingbayesianda_solutions_directory = "Krushcke J.K. DoingBayesian DataAnalysis. Solutions.pdf"

In [41]:
def read_pdf(pdf_path):
    """
    Читает PDF-файл и извлекает текст по главам на основе закладок.
    
    Аргументы:
    - pdf_path: путь к PDF-файлу.
    
    Возвращает:
    - main_chapters: словарь, где ключом является название главы, а значением – текст этой главы.
    """
    
    doc = fitz.open(pdf_path)         # Открываем PDF-документ
    bookmarks = doc.get_toc()           # Получаем оглавление (закладки)
    
    main_chapters = {}                # Словарь для хранения глав
    current_main_chapter = None       # Текущая основная глава
    
    # Проходим по всем закладкам
    for i, bookmark in enumerate(bookmarks):
        level, title, page_start = bookmark 
        page_start -= 1  # Переводим страницу в 0-индексацию
        
        # Определяем конец главы: до следующей закладки или до конца документа
        page_end = bookmarks[i + 1][2] - 1 if i + 1 < len(bookmarks) else len(doc)
        
        chapter_text = ""
        # Собираем текст со страниц от начала до конца главы
        for page_num in range(page_start, page_end):
            chapter_text += doc.load_page(page_num).get_text()
        
        # Если уровень закладки равен 1, это основная глава
        if level == 1:
            current_main_chapter = title
            main_chapters[current_main_chapter] = chapter_text.strip()
        # Если это подпункт, добавляем его к текущей главе
        elif current_main_chapter:
            main_chapters[current_main_chapter] += "\n\n" + chapter_text.strip()
    
    return main_chapters

In [45]:
# Читаем книгу Ричарда МакЭлрита
rethinking_raw = read_pdf(rethinking_directory)

# Определяем названия глав, которые необходимо исключить
rethinking_chapters_exclusion = [
    'Cover', 
    'Half Title', 
    'Title Page', 
    'Copyright Page', 
    'Table of Contents', 
    'Preface to The Second Edition', 
    'Preface', 
    'Endnotes', 
    'Bibliography', 
    'Citation Index', 
    'Topic Index'
]

# Фильтруем главы, оставляя только содержательные
rethinking_raw = {title: text for title, text in rethinking_raw.items() if title not in rethinking_chapters_exclusion}

# Записываем результаты в DataFrame для дальнейшей обработки
rethinking_df = pd.DataFrame(list(rethinking_raw.items()), columns=["chapter", "text"])
rethinking_df['source'] = 'McElreath R.'  # Добавляем информацию об авторе

# Выводим DataFrame для проверки результата
rethinking_df

,chapter,text,source
0,Chapter 1: The Golem of Prague,\n\n1 The Golem of Prague\nIn the sixteenth ce...,McElreath R.
1,Chapter 2: Small Worlds and Large Worlds,2 Small Worlds and Large Worlds\nWhen Cristofo...,McElreath R.
2,Chapter 3: Sampling the Imaginary,3 Sampling the Imaginary\nLots of books on Bay...,McElreath R.
3,Chapter 4: Geocentric Models,4 Geocentric Models\nHistory has been unkind t...,McElreath R.
4,Chapter 5: The Many Variables & The Spurious W...,5 The Many Variables & The Spurious Waffles\nO...,McElreath R.
5,Chapter 6: The Haunted Dag & The Causal Terror,6 The Haunted DAG & The Causal Terror\nIt seem...,McElreath R.
6,Chapter 7: Ulysses' Compass,7 Ulysses’ Compass\nMikołaj Kopernik (also kno...,McElreath R.
7,Chapter 8: Conditional Manatees,8 Conditional Manatees\nThe manatee (Trichechu...,McElreath R.
8,Chapter 9: Markov Chain Monte Carlo,9 Markov Chain Monte Carlo\nIn the twentieth c...,McElreath R.
9,Chapter 10: Big Entropy and the Generalized Li...,10 Big Entropy and the Generalized Linear Mode...,McElreath R.


In [47]:
# Читаем книгу Джона Крушке
doingbayesianda_raw = read_pdf(doingbayesianda_directory)

# Определяем названия глав, которые необходимо исключить
doingbayesianda_chapters_exclusion = [
    "1. Front-Matter_2015_Doing-Bayesian-Data-Analysis-Second-Edition-",
    "2. Copyright_2015_Doing-Bayesian-Data-Analysis-Second-Edition-",
    "3. Dedication_2015_Doing-Bayesian-Data-Analysis-Second-Edition-",
    "32. Bibliography_2015_Doing-Bayesian-Data-Analysis-Second-Edition-",
    "33. Index_2015_Doing-Bayesian-Data-Analysis-Second-Edition-"
]

# Фильтруем главы
doingbayesianda_raw = {
    title: text 
    for title, text in doingbayesianda_raw.items() 
    if title not in doingbayesianda_chapters_exclusion
}

# Записываем результаты в DataFrame
doingbayesianda_df = pd.DataFrame(list(doingbayesianda_raw.items()), columns=["chapter", "text"])
doingbayesianda_df['source'] = 'Krushcke J.K.' # Добавляем информацию об авторе

# Выводим DataFrame для проверки результата
doingbayesianda_df

,chapter,text,source
0,4. Chapter-1-What-s-in-This-Book-Read-This-Fir...,\n\n\n\nCHAPTER 1\nWhat’s in This Book (Read T...,Krushcke J.K.
1,5. Part-I-Introduction_2015_Doing-Bayesian-Dat...,"\n\nPART I\nThe Basics: Models,\nProbability, ...",Krushcke J.K.
2,6. Chapter-2-Introduction-Credibility-Models-a...,"\n\nCHAPTER 2\nIntroduction: Credibility, Mode...",Krushcke J.K.
3,7. Chapter-3-The-R-Programming-Language_2015_D...,\n\nCHAPTER 3\nThe R Programming Language\nCon...,Krushcke J.K.
4,8. Chapter-4-What-is-This-Stuff-Called-Probabi...,\n\nCHAPTER 4\nWhat Is This Stuff Called Proba...,Krushcke J.K.
5,9. Chapter-5-Bayes-Rule_2015_Doing-Bayesian-Da...,\n\nCHAPTER 5\nBayes’Rule\nContents\n5.1. Baye...,Krushcke J.K.
6,10. Part-II-Introduction_2015_Doing-Bayesian-D...,\n\nPART II\nAll the Fundamentals\nApplied to ...,Krushcke J.K.
7,11. Chapter-6-Inferring-a-Binomial-Probability...,\n\nCHAPTER 6\nInferring a Binomial Probabilit...,Krushcke J.K.
8,12. Chapter-7-Markov-Chain-Monte-Carlo_2015_Do...,\n\nCHAPTER 7\nMarkov Chain Monte Carlo\nConte...,Krushcke J.K.
9,13. Chapter-8-JAGS_2015_Doing-Bayesian-Data-An...,\n\n\n\nCHAPTER 8\nJAGS\nContents\n8.1. JAGS a...,Krushcke J.K.


In [49]:
def pretty_chapter_title(input_string):
    """
    Приводит название главы к более читабельному виду.
    
    Аргументы:
    - input_string: исходная строка с названием главы.
    
    Возвращает:
    - cleaned_string: очищенная и форматированная строка.
    """
    # Удаляем ведущие цифры и точку (например, "1. ")
    cleaned_string = re.sub(r"^\d+\.\s*", "", input_string)
    # Удаляем всё, что идёт после символа подчеркивания (например, "_2015_...")
    cleaned_string = re.sub(r'_.*$', '', cleaned_string)
    # Заменяем дефисы на пробелы
    cleaned_string = cleaned_string.replace("-", " ")
    # Добавляем двоеточие после слова "Chapter" и номера
    cleaned_string = re.sub(r"Chapter (\d+)", r"Chapter \1:", cleaned_string)
    # Исправляем возможное отсутствие апострофа (например, "John s" -> "John's")
    cleaned_string = re.sub(r"(\w) s ", r"\1's ", cleaned_string)
    return cleaned_string.strip()

# Применяем функцию к столбцу с названиями глав в DataFrame книги Крушке
doingbayesianda_df['chapter'] = doingbayesianda_df['chapter'].apply(pretty_chapter_title)

# Выводим DataFrame для проверки результата
doingbayesianda_df

,chapter,text,source
0,Chapter 1: What's in This Book Read This First,\n\n\n\nCHAPTER 1\nWhat’s in This Book (Read T...,Krushcke J.K.
1,Part I Introduction,"\n\nPART I\nThe Basics: Models,\nProbability, ...",Krushcke J.K.
2,Chapter 2: Introduction Credibility Models and...,"\n\nCHAPTER 2\nIntroduction: Credibility, Mode...",Krushcke J.K.
3,Chapter 3: The R Programming Language,\n\nCHAPTER 3\nThe R Programming Language\nCon...,Krushcke J.K.
4,Chapter 4: What is This Stuff Called Probability,\n\nCHAPTER 4\nWhat Is This Stuff Called Proba...,Krushcke J.K.
5,Chapter 5: Bayes Rule,\n\nCHAPTER 5\nBayes’Rule\nContents\n5.1. Baye...,Krushcke J.K.
6,Part II Introduction,\n\nPART II\nAll the Fundamentals\nApplied to ...,Krushcke J.K.
7,Chapter 6: Inferring a Binomial Probability vi...,\n\nCHAPTER 6\nInferring a Binomial Probabilit...,Krushcke J.K.
8,Chapter 7: Markov Chain Monte Carlo,\n\nCHAPTER 7\nMarkov Chain Monte Carlo\nConte...,Krushcke J.K.
9,Chapter 8: JAGS,\n\n\n\nCHAPTER 8\nJAGS\nContents\n8.1. JAGS a...,Krushcke J.K.


In [51]:
def read_pdf_by_pages(pdf_path, chapter_ranges):
    """
    Читает PDF-файл и извлекает текст по заданным диапазонам страниц для каждого раздела.
    
    Аргументы:
    - pdf_path: путь к PDF-файлу.
    - chapter_ranges: список кортежей, где каждый кортеж содержит:
        (название раздела, номер стартовой страницы, номер конечной страницы).
        Нумерация страниц начинается с 1.
    
    Возвращает:
    - main_chapters: словарь, где ключ — название раздела, а значение — текст, извлечённый из указанного диапазона страниц.
    """
    
    # Открываем PDF-документ с помощью PyMuPDF (fitz)
    doc = fitz.open(pdf_path)
    main_chapters = {}  # Словарь для хранения результатов
    
    # Проходим по каждому разделу, определённому в chapter_ranges
    for chapter_title, start_page, end_page in chapter_ranges:
        chapter_text = ""  # Инициализируем переменную для накопления текста раздела
        # Обратите внимание: страницы в PyMuPDF индексируются с 0, поэтому вычитаем 1 из start_page.
        for page_num in range(start_page - 1, end_page):
            # Получаем текст со страницы, удаляем лишние пробелы и добавляем символ перевода строки для разделения страниц
            text = doc.load_page(page_num).get_text()
            chapter_text += text.strip() + "\n"
        # Сохраняем текст раздела в словаре с ключом chapter_title
        main_chapters[chapter_title] = chapter_text.strip()
    
    return main_chapters

# Задаём диапазоны страниц для каждой главы
chapter_ranges = [
    ("Chapter 2. Solutions", 5, 6),
    ("Chapter 3. Solutions", 7, 9),
    ("Chapter 4. Solutions", 10, 16),
    ("Chapter 5. Solutions", 17, 31),
    ("Chapter 6. Solutions", 32, 41),
    ("Chapter 7. Solutions", 42, 51),
    ("Chapter 8. Solutions", 52, 58),
    ("Chapter 9. Solutions", 59, 70),
    ("Chapter 10. Solutions", 71, 80),
    ("Chapter 11. Solutions", 81, 86),
    ("Chapter 12. Solutions", 87, 94),
    ("Chapter 13. Solutions", 95, 104),
    ("Chapter 14. Solutions", 105, 110),
    ("Chapter 15. Solutions", 111, 113),
    ("Chapter 16. Solutions", 114, 122),
    ("Chapter 17. Solutions", 123, 131),
    ("Chapter 18. Solutions", 132, 141)
]

# Читаем PDF-файл с решениями к книге Джона Крушке, используя указанные диапазоны страниц
doingbayesianda_solutions_raw = read_pdf_by_pages(doingbayesianda_solutions_directory, chapter_ranges)

# Преобразуем полученный словарь в DataFrame для удобного анализа и дальнейшей обработки
doingbayesianda_solutions_df = pd.DataFrame(list(doingbayesianda_solutions_raw.items()), columns=["chapter", "text"])
doingbayesianda_solutions_df['source'] = 'Krushcke J.K.' # Добавляем информацию об авторе

# Выводим DataFrame для проверки результата
doingbayesianda_solutions_df

,chapter,text,source
0,Chapter 2. Solutions,Solutions to Exercises in Doing Bayesian Data ...,Krushcke J.K.
1,Chapter 3. Solutions,Solutions to Exercises in Doing Bayesian Data ...,Krushcke J.K.
2,Chapter 4. Solutions,Solutions to Exercises in Doing Bayesian Data ...,Krushcke J.K.
3,Chapter 5. Solutions,Solutions to Exercises in Doing Bayesian Data ...,Krushcke J.K.
4,Chapter 6. Solutions,Solutions to Exercises in Doing Bayesian Data ...,Krushcke J.K.
5,Chapter 7. Solutions,Solutions to Exercises in Doing Bayesian Data ...,Krushcke J.K.
6,Chapter 8. Solutions,Solutions to Exercises in Doing Bayesian Data ...,Krushcke J.K.
7,Chapter 9. Solutions,Solutions to Exercises in Doing Bayesian Data ...,Krushcke J.K.
8,Chapter 10. Solutions,Solutions to Exercises in Doing Bayesian Data ...,Krushcke J.K.
9,Chapter 11. Solutions,Solutions to Exercises in Doing Bayesian Data ...,Krushcke J.K.


In [53]:
# Определяем регулярное выражение для удаления колонтитулов,
# в котором содержатся фрагменты: название книги, год, нумерация страниц и номера глав.
footnote_pattern = r"(Solutions to Exercises in Doing Bayesian Data Analysis 2nd Ed\. by John K\. Kruschke © 2015\.\s*Page \d+ of \d+\s*Chapter(s)? \d+)"

# Удаляем найденные шаблоны из столбца 'text'
doingbayesianda_solutions_df["text"] = doingbayesianda_solutions_df["text"].str.replace(footnote_pattern, "", regex=True)

In [ ]:
# Загружаем данные из объединённого DataFrame с нашими книгами.
loader = DataFrameLoader(books, page_content_column="text")
data = loader.load()

# Разбиваем документы на чанки для лучшей генерации эмбеддингов и поиска релевантного контекста
chunk_overlap — перекрытие между чанками.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.split_documents(data)
len(docs)
# 4135

# Определяем путь для хранения локальной базы данных Chroma.
CHROMA_DATA_PATH = "chroma_data_bayesium/"
# Используем модель для генерации эмбеддингов.
EMBED_MODEL = "all-MiniLM-L6-v2"


# Создаем клиент ChromaDB, чтобы данные сохранялись локально.
chromadb_client = chromadb.PersistentClient(path=CHROMA_DATA_PATH)

# Задаем имя коллекции, в которой будут храниться текстовые чанки наших книг.
COLLECTION_NAME = "bayesium"

# Определяем embedding_function на базе модели SentenceTransformer.
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=EMBED_MODEL
)

# Создаем коллекцию в ChromaDB. Обратите внимание, что в metadata указываем использование косинусной меры, что улучшает точность поиска схожих эмбеддингов.
collection = chromadb_client.create_collection(
    name=COLLECTION_NAME,
    embedding_function=embedding_func,
    metadata={"hnsw:space": "cosine"},
)

# Определяем метаданные 
# В метадате передается информация об авторе, названии главы и индексе чанка.
custom_metadata = [
    {
        'source': doc.metadata.get('source', 'Unknown'),
        'chapter': doc.metadata.get('chapter', 'Unknown'),
        'chunk_index': index
    } 
    for index, doc in enumerate(docs)
]

# Далее мы записываем полученные текстовые чанки в базу данных батчами. 
# Функция create_batches автоматически разбивает данные на группы, которые потом загружаются в коллекцию.
batches = create_batches(
    api=chromadb_client,
    ids=[f"{uuid.uuid4()}" for i in range(len(docs))], 
    documents=[doc.page_content for doc in docs], 
    metadatas=custom_metadata  # Use the customized metadata
)

# Добавляем данные в коллекцию батчами. 
for batch in batches:
    print(f"Adding batch of size {len(batch[0])}")
    collection.add(
        ids=batch[0],     
        documents=batch[3], 
        metadatas=batch[2] 
    )

In [59]:
question = "How to use ulam()?"

collection.query(
        query_texts=[question],
        n_results=5
    )

{'ids': [['f4004c4f-96db-4b88-92a7-5e7d018f377a',
   'd753eeec-d216-4f47-ad61-3aa5158b86cd',
   '173d8477-141e-4fe9-8367-7fdbe76d35e4',
   'e2c040f1-f3d0-41ac-b681-77cb17deb982',
   '4a2eae91-6829-4765-b91d-ac471a678473']],
 'embeddings': None,
 'documents': [['defined in the section above. Let’s focus on how this is implemented in Stan. When you tell ulam to\nuse dzipois, it understands it like this:\nR code\n12.11\nm12.3_alt <- ulam(\nalist(\ny|y>0 ~ custom( log1m(p) + poisson_lpmf(y|lambda) ),\ny|y==0 ~ custom( log_mix( p , 0 , poisson_lpmf(0|lambda) ) ),\nlogit(p) <- ap,\nlog(lambda) <- al,\nap ~ dnorm(-1.5,1),',
   'You should also see several warning messages, including:\nWarning messages:\n1: There were 67 divergent transitions after warmup. Increasing adapt_delta\nabove 0.95 may help. See\nhttp://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup\nThere is useful advice at the URL. The quick version is that Stan detected problems in ex-\nploring all of the poster